In [ ]:
# ⚙️ Setup
import subprocess, sys
try:
    import google.colab
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "numba"])
except ImportError:
    pass

import numpy as np
from numba import cuda

print("⚠️  CUDA C++ is PRIMARY. Python/Numba for quick testing only.")
print("⚠️  Multi-GPU patterns require multiple physical GPUs!")
if cuda.is_available():
    print(f"GPU: {cuda.get_current_device().name}")

# Day 2: Multi-GPU Patterns - Manager Coordinating Multiple Factories

## 🎯 The Hook: One Manager, Many Factories!

**You've been promoted from warehouse manager to regional director!** You now oversee multiple factories, each producing parts of a larger product. Your challenge: **coordinate them to work as one efficient system**.

Consider building a car:
- **Factory 0**: Builds the front half (engine, headlights, hood)
- **Factory 1**: Builds the rear half (trunk, taillights, bumper)
- **The assembly line**: Needs parts from BOTH to create the final product

But wait - what about the **middle of the car** where front meets rear? Both factories need to know about the boundaries to make parts that fit together perfectly!

**This is multi-GPU programming at scale:**
- **Domain decomposition**: Split the problem across GPUs
- **Halo exchange**: Share boundary information
- **Reduction patterns**: Combine partial results

Today you'll learn to think like a regional director managing multiple factories! 🏭🏭🏭

---

## 🎯 Learning Objectives

By the end of this notebook, you will be able to:

1. **Implement domain decomposition** - Split arrays and matrices across GPUs evenly
2. **Handle boundary conditions** - Exchange halo cells for stencil operations
3. **Perform multi-GPU reductions** - Combine partial sums from each GPU efficiently
4. **Choose load balancing strategies** - Static vs dynamic work distribution
5. **Design scalable patterns** - Write code that works for 2, 4, or 8 GPUs

---

## 🎴 Concept Card: Manager Coordinating Multiple Factories

```
┌─────────────────────────────────────────────────────────────────┐
│            MANAGER COORDINATING MULTIPLE FACTORIES              │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│  PRODUCTION FLOW:                                               │
│  ════════════════                                               │
│                                                                 │
│        ┌──────────────────────────────────────┐                 │
│        │         FULL PRODUCT LINE            │                 │
│        │    [Parts 0-3 | Parts 4-7 | 8-11]    │                 │
│        └───────┬────────────┬───────────┬─────┘                 │
│                │            │           │                       │
│           ┌────▼────┐  ┌────▼────┐  ┌───▼────┐                  │
│           │ Factory │  │ Factory │  │ Factory│                  │
│           │    0    │  │    1    │  │    2   │                  │
│           │[0,1,2,3]│  │[4,5,6,7]│  │[8-11]  │                  │
│           └────┬────┘  └────┬────┘  └───┬────┘                  │
│                │  Boundary  │           │                       │
│                │◄──Share──►│◄──Share──►│                        │
│                │  (Halo)   │  (Halo)   │                        │
│           ┌────▼────┐  ┌────▼────┐  ┌───▼────┐                  │
│           │ Partial │  │ Partial │  │Partial │                  │
│           │ Sum: 10 │  │ Sum: 26 │  │Sum: 38 │                  │
│           └────┬────┘  └────┬────┘  └───┬────┘                  │
│                └─────────┬──┴───────────┘                       │
│                          ▼                                      │
│                  ┌──────────────┐                               │
│                  │ FINAL: 74    │ ← Reduction                   │
│                  │ (Combined)   │                               │
│                  └──────────────┘                               │
│                                                                 │
├─────────────────────────────────────────────────────────────────┤
│  PATTERNS:                                                      │
│                                                                 │
│  1️⃣ DOMAIN DECOMPOSITION:                                      │
│     N elements ÷ K GPUs = chunk per GPU                         │
│     Handle remainder: GPU i gets extra if i < (N % K)           │
│                                                                 │
│  2️⃣ HALO EXCHANGE (for stencils):                              │
│     Before compute: get neighbor's boundary                     │
│     [ghost|real data...|ghost]                                  │
│                                                                 │
│  3️⃣ REDUCTION:                                                  │
│     Each GPU: local reduce → partial result                     │
│     Manager: combine partials → final result                    │
│                                                                 │
├─────────────────────────────────────────────────────────────────┤
│  💡 KEY INSIGHT:                                                │
│  Good managers (you!) minimize communication between factories  │
│  while ensuring everyone has what they need. Same with GPUs -   │
│  minimize P2P transfers, maximize independent computation!      │
└─────────────────────────────────────────────────────────────────┘
```

---

## Part 1: Domain Decomposition

### Splitting Work Across GPUs

```
1D Domain Decomposition:
━━━━━━━━━━━━━━━━━━━━━━━━━

Full Array: [0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15]
                    ↓
GPU 0:      [0 1 2 3 4 5 6 7]
GPU 1:      [8 9 10 11 12 13 14 15]

2D Domain Decomposition:
━━━━━━━━━━━━━━━━━━━━━━━━━

┌─────────┬─────────┐
│  GPU 0  │  GPU 1  │
├─────────┼─────────┤
│  GPU 2  │  GPU 3  │
└─────────┴─────────┘
```

### 🔷 CUDA C++ Domain Decomposition (Primary)

```cpp
// domain_decomp.cu - 1D domain decomposition
#include <stdio.h>
#include <cuda_runtime.h>

__global__ void processChunk(float* data, int n, int globalOffset) {
    int tid = blockIdx.x * blockDim.x + threadIdx.x;
    if (tid < n) {
        int globalIdx = globalOffset + tid;
        data[tid] = sinf((float)globalIdx * 0.01f);
    }
}

int main() {
    int deviceCount;
    cudaGetDeviceCount(&deviceCount);
    
    const int TOTAL_N = 1 << 24;  // 16M elements
    const int NUM_GPUS = min(deviceCount, 4);
    
    printf("Using %d GPUs for %d elements\n", NUM_GPUS, TOTAL_N);
    
    // Calculate chunk sizes
    int baseChunkSize = TOTAL_N / NUM_GPUS;
    int remainder = TOTAL_N % NUM_GPUS;
    
    // Arrays for per-GPU data
    float* d_data[NUM_GPUS];
    int chunkSizes[NUM_GPUS];
    int offsets[NUM_GPUS];
    cudaStream_t streams[NUM_GPUS];
    
    // ============================================
    // Calculate Per-GPU Chunks (Handle Remainder)
    // ============================================
    int currentOffset = 0;
    for (int gpu = 0; gpu < NUM_GPUS; gpu++) {
        // Distribute remainder across first few GPUs
        chunkSizes[gpu] = baseChunkSize + (gpu < remainder ? 1 : 0);
        offsets[gpu] = currentOffset;
        currentOffset += chunkSizes[gpu];
        
        printf("GPU %d: offset=%d, size=%d\n", 
               gpu, offsets[gpu], chunkSizes[gpu]);
    }
    
    // ============================================
    // Allocate on Each GPU
    // ============================================
    for (int gpu = 0; gpu < NUM_GPUS; gpu++) {
        cudaSetDevice(gpu);
        cudaMalloc(&d_data[gpu], chunkSizes[gpu] * sizeof(float));
        cudaStreamCreate(&streams[gpu]);
    }
    
    // ============================================
    // Launch Kernels on Each GPU
    // ============================================
    int blockSize = 256;
    
    for (int gpu = 0; gpu < NUM_GPUS; gpu++) {
        cudaSetDevice(gpu);
        int numBlocks = (chunkSizes[gpu] + blockSize - 1) / blockSize;
        
        processChunk<<<numBlocks, blockSize, 0, streams[gpu]>>>(
            d_data[gpu], chunkSizes[gpu], offsets[gpu]);
    }
    
    // ============================================
    // Synchronize All GPUs
    // ============================================
    for (int gpu = 0; gpu < NUM_GPUS; gpu++) {
        cudaSetDevice(gpu);
        cudaStreamSynchronize(streams[gpu]);
    }
    
    printf("All GPUs finished processing!\n");
    
    // Cleanup
    for (int gpu = 0; gpu < NUM_GPUS; gpu++) {
        cudaSetDevice(gpu);
        cudaFree(d_data[gpu]);
        cudaStreamDestroy(streams[gpu]);
    }
    
    return 0;
}
```

### 🔷 CUDA C++ Implementation (Primary)

In [ ]:
%%writefile domain_decomp.cu
// domain_decomp.cu - 1D domain decomposition
#include <stdio.h>
#include <cuda_runtime.h>

__global__ void processChunk(float* data, int n, int globalOffset) {
    int tid = blockIdx.x * blockDim.x + threadIdx.x;
    if (tid < n) {
        int globalIdx = globalOffset + tid;
        data[tid] = sinf((float)globalIdx * 0.01f);
    }
}

int main() {
    int deviceCount;
    cudaGetDeviceCount(&deviceCount);
    
    const int TOTAL_N = 1 << 24;  // 16M elements
    const int NUM_GPUS = min(deviceCount, 4);
    
    printf("Using %d GPUs for %d elements\n", NUM_GPUS, TOTAL_N);
    
    // Calculate chunk sizes
    int baseChunkSize = TOTAL_N / NUM_GPUS;
    int remainder = TOTAL_N % NUM_GPUS;
    
    // Arrays for per-GPU data
    float* d_data[NUM_GPUS];
    int chunkSizes[NUM_GPUS];
    int offsets[NUM_GPUS];
    cudaStream_t streams[NUM_GPUS];
    
    // ============================================
    // Calculate Per-GPU Chunks (Handle Remainder)
    // ============================================
    int currentOffset = 0;
    for (int gpu = 0; gpu < NUM_GPUS; gpu++) {
        // Distribute remainder across first few GPUs
        chunkSizes[gpu] = baseChunkSize + (gpu < remainder ? 1 : 0);
        offsets[gpu] = currentOffset;
        currentOffset += chunkSizes[gpu];
        
        printf("GPU %d: offset=%d, size=%d\n", 
               gpu, offsets[gpu], chunkSizes[gpu]);
    }
    
    // ============================================
    // Allocate on Each GPU
    // ============================================
    for (int gpu = 0; gpu < NUM_GPUS; gpu++) {
        cudaSetDevice(gpu);
        cudaMalloc(&d_data[gpu], chunkSizes[gpu] * sizeof(float));
        cudaStreamCreate(&streams[gpu]);
    }
    
    // ============================================
    // Launch Kernels on Each GPU
    // ============================================
    int blockSize = 256;
    
    for (int gpu = 0; gpu < NUM_GPUS; gpu++) {
        cudaSetDevice(gpu);
        int numBlocks = (chunkSizes[gpu] + blockSize - 1) / blockSize;
        
        processChunk<<<numBlocks, blockSize, 0, streams[gpu]>>>(
            d_data[gpu], chunkSizes[gpu], offsets[gpu]);
    }
    
    // ============================================
    // Synchronize All GPUs
    // ============================================
    for (int gpu = 0; gpu < NUM_GPUS; gpu++) {
        cudaSetDevice(gpu);
        cudaStreamSynchronize(streams[gpu]);
    }
    
    printf("All GPUs finished processing!\n");
    
    // Cleanup
    for (int gpu = 0; gpu < NUM_GPUS; gpu++) {
        cudaSetDevice(gpu);
        cudaFree(d_data[gpu]);
        cudaStreamDestroy(streams[gpu]);
    }
    
    return 0;
}

In [ ]:
!nvcc -arch=sm_75 -o domain_decomp domain_decomp.cu
!./domain_decomp

---

## Part 2: Halo Exchange

### Stencil Operations Require Boundary Data

```
Problem: Stencil needs neighbors
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

GPU 0 data:  [a b c d e | f g h]
GPU 1 data:  [i j k | l m n o p]
               ↑       ↑
         Need f,g,h  Need e

Solution: Halo/Ghost cells
━━━━━━━━━━━━━━━━━━━━━━━━━━

GPU 0: [a b c d e | f g h] + [i j k]  ← halo from GPU 1
GPU 1: [f g h] + [i j k | l m n o p]  ← halo from GPU 0
        ↑                     
   halo from GPU 0
```

### 🔷 CUDA C++ Halo Exchange (Primary)

```cpp
// halo_exchange.cu - Exchange boundary data between GPUs
#include <stdio.h>
#include <cuda_runtime.h>

#define HALO_SIZE 3  // Stencil radius

// Stencil kernel (5-point average)
__global__ void stencil(float* out, float* in, int n) {
    int tid = blockIdx.x * blockDim.x + threadIdx.x;
    
    // Skip halo regions for output
    if (tid >= HALO_SIZE && tid < n - HALO_SIZE) {
        out[tid] = 0.2f * (in[tid-2] + in[tid-1] + in[tid] + 
                          in[tid+1] + in[tid+2]);
    }
}

int main() {
    int deviceCount;
    cudaGetDeviceCount(&deviceCount);
    
    if (deviceCount < 2) {
        printf("Need 2 GPUs for halo exchange\n");
        return 1;
    }
    
    const int CHUNK_SIZE = 1024;  // Data per GPU
    const int TOTAL_SIZE = CHUNK_SIZE + 2 * HALO_SIZE;  // + halos
    
    float *d_in[2], *d_out[2];
    cudaStream_t streams[2];
    
    // Enable P2P
    int canP2P;
    cudaDeviceCanAccessPeer(&canP2P, 0, 1);
    if (canP2P) {
        cudaSetDevice(0); cudaDeviceEnablePeerAccess(1, 0);
        cudaSetDevice(1); cudaDeviceEnablePeerAccess(0, 0);
    }
    
    // Allocate with halo space
    for (int gpu = 0; gpu < 2; gpu++) {
        cudaSetDevice(gpu);
        cudaMalloc(&d_in[gpu], TOTAL_SIZE * sizeof(float));
        cudaMalloc(&d_out[gpu], TOTAL_SIZE * sizeof(float));
        cudaStreamCreate(&streams[gpu]);
        
        // Initialize
        cudaMemset(d_in[gpu], 0, TOTAL_SIZE * sizeof(float));
    }
    
    // ============================================
    // Halo Exchange
    // ============================================
    // GPU 0's right boundary -> GPU 1's left halo
    // GPU 1's left boundary -> GPU 0's right halo
    
    // GPU 0: Copy rightmost data to GPU 1's left halo
    // d_in[0][HALO_SIZE + CHUNK_SIZE - HALO_SIZE ... ] -> d_in[1][0...]
    cudaMemcpyPeerAsync(
        d_in[1],                    // dst: GPU 1's left halo
        1,                          // dst device
        d_in[0] + CHUNK_SIZE,       // src: GPU 0's right boundary
        0,                          // src device
        HALO_SIZE * sizeof(float),
        streams[0]);
    
    // GPU 1: Copy leftmost data to GPU 0's right halo
    cudaMemcpyPeerAsync(
        d_in[0] + HALO_SIZE + CHUNK_SIZE,  // dst: GPU 0's right halo
        0,
        d_in[1] + HALO_SIZE,               // src: GPU 1's left boundary
        1,
        HALO_SIZE * sizeof(float),
        streams[1]);
    
    // Sync before compute
    cudaSetDevice(0); cudaStreamSynchronize(streams[0]);
    cudaSetDevice(1); cudaStreamSynchronize(streams[1]);
    
    // ============================================
    // Compute Stencil
    // ============================================
    int blockSize = 256;
    int numBlocks = (TOTAL_SIZE + blockSize - 1) / blockSize;
    
    for (int gpu = 0; gpu < 2; gpu++) {
        cudaSetDevice(gpu);
        stencil<<<numBlocks, blockSize, 0, streams[gpu]>>>(
            d_out[gpu], d_in[gpu], TOTAL_SIZE);
    }
    
    // Sync
    for (int gpu = 0; gpu < 2; gpu++) {
        cudaSetDevice(gpu);
        cudaStreamSynchronize(streams[gpu]);
    }
    
    printf("Halo exchange and stencil complete!\n");
    
    // Cleanup
    for (int gpu = 0; gpu < 2; gpu++) {
        cudaSetDevice(gpu);
        cudaFree(d_in[gpu]);
        cudaFree(d_out[gpu]);
        cudaStreamDestroy(streams[gpu]);
    }
    
    return 0;
}
```

### 🔷 CUDA C++ Implementation (Primary)

In [ ]:
%%writefile halo_exchange.cu
// halo_exchange.cu - Exchange boundary data between GPUs
#include <stdio.h>
#include <cuda_runtime.h>

#define HALO_SIZE 3  // Stencil radius

// Stencil kernel (5-point average)
__global__ void stencil(float* out, float* in, int n) {
    int tid = blockIdx.x * blockDim.x + threadIdx.x;
    
    // Skip halo regions for output
    if (tid >= HALO_SIZE && tid < n - HALO_SIZE) {
        out[tid] = 0.2f * (in[tid-2] + in[tid-1] + in[tid] + 
                          in[tid+1] + in[tid+2]);
    }
}

int main() {
    int deviceCount;
    cudaGetDeviceCount(&deviceCount);
    
    if (deviceCount < 2) {
        printf("Need 2 GPUs for halo exchange\n");
        return 1;
    }
    
    const int CHUNK_SIZE = 1024;  // Data per GPU
    const int TOTAL_SIZE = CHUNK_SIZE + 2 * HALO_SIZE;  // + halos
    
    float *d_in[2], *d_out[2];
    cudaStream_t streams[2];
    
    // Enable P2P
    int canP2P;
    cudaDeviceCanAccessPeer(&canP2P, 0, 1);
    if (canP2P) {
        cudaSetDevice(0); cudaDeviceEnablePeerAccess(1, 0);
        cudaSetDevice(1); cudaDeviceEnablePeerAccess(0, 0);
    }
    
    // Allocate with halo space
    for (int gpu = 0; gpu < 2; gpu++) {
        cudaSetDevice(gpu);
        cudaMalloc(&d_in[gpu], TOTAL_SIZE * sizeof(float));
        cudaMalloc(&d_out[gpu], TOTAL_SIZE * sizeof(float));
        cudaStreamCreate(&streams[gpu]);
        
        // Initialize
        cudaMemset(d_in[gpu], 0, TOTAL_SIZE * sizeof(float));
    }
    
    // ============================================
    // Halo Exchange
    // ============================================
    // GPU 0's right boundary -> GPU 1's left halo
    // GPU 1's left boundary -> GPU 0's right halo
    
    // GPU 0: Copy rightmost data to GPU 1's left halo
    // d_in[0][HALO_SIZE + CHUNK_SIZE - HALO_SIZE ... ] -> d_in[1][0...]
    cudaMemcpyPeerAsync(
        d_in[1],                    // dst: GPU 1's left halo
        1,                          // dst device
        d_in[0] + CHUNK_SIZE,       // src: GPU 0's right boundary
        0,                          // src device
        HALO_SIZE * sizeof(float),
        streams[0]);
    
    // GPU 1: Copy leftmost data to GPU 0's right halo
    cudaMemcpyPeerAsync(
        d_in[0] + HALO_SIZE + CHUNK_SIZE,  // dst: GPU 0's right halo
        0,
        d_in[1] + HALO_SIZE,               // src: GPU 1's left boundary
        1,
        HALO_SIZE * sizeof(float),
        streams[1]);
    
    // Sync before compute
    cudaSetDevice(0); cudaStreamSynchronize(streams[0]);
    cudaSetDevice(1); cudaStreamSynchronize(streams[1]);
    
    // ============================================
    // Compute Stencil
    // ============================================
    int blockSize = 256;
    int numBlocks = (TOTAL_SIZE + blockSize - 1) / blockSize;
    
    for (int gpu = 0; gpu < 2; gpu++) {
        cudaSetDevice(gpu);
        stencil<<<numBlocks, blockSize, 0, streams[gpu]>>>(
            d_out[gpu], d_in[gpu], TOTAL_SIZE);
    }
    
    // Sync
    for (int gpu = 0; gpu < 2; gpu++) {
        cudaSetDevice(gpu);
        cudaStreamSynchronize(streams[gpu]);
    }
    
    printf("Halo exchange and stencil complete!\n");
    
    // Cleanup
    for (int gpu = 0; gpu < 2; gpu++) {
        cudaSetDevice(gpu);
        cudaFree(d_in[gpu]);
        cudaFree(d_out[gpu]);
        cudaStreamDestroy(streams[gpu]);
    }
    
    return 0;
}

In [ ]:
!nvcc -arch=sm_75 -o halo_exchange halo_exchange.cu
!./halo_exchange

---

## Part 3: Multi-GPU Reduction

### Combining Results Across GPUs

```cpp
// multi_gpu_reduction.cu - Reduce across multiple GPUs
#include <stdio.h>
#include <cuda_runtime.h>

// Block reduction kernel
__global__ void reduce(float* out, float* in, int n) {
    extern __shared__ float sdata[];
    
    int tid = threadIdx.x;
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    
    sdata[tid] = (idx < n) ? in[idx] : 0.0f;
    __syncthreads();
    
    for (int s = blockDim.x / 2; s > 0; s >>= 1) {
        if (tid < s) {
            sdata[tid] += sdata[tid + s];
        }
        __syncthreads();
    }
    
    if (tid == 0) {
        out[blockIdx.x] = sdata[0];
    }
}

int main() {
    int deviceCount;
    cudaGetDeviceCount(&deviceCount);
    const int NUM_GPUS = min(deviceCount, 4);
    
    const int TOTAL_N = 1 << 24;
    const int N_PER_GPU = TOTAL_N / NUM_GPUS;
    const int BLOCK_SIZE = 256;
    const int NUM_BLOCKS = (N_PER_GPU + BLOCK_SIZE - 1) / BLOCK_SIZE;
    
    // Per-GPU data
    float* d_data[NUM_GPUS];
    float* d_partial[NUM_GPUS];
    float h_partial[NUM_GPUS];  // Final partial sums
    cudaStream_t streams[NUM_GPUS];
    
    // Allocate and initialize
    for (int gpu = 0; gpu < NUM_GPUS; gpu++) {
        cudaSetDevice(gpu);
        cudaMalloc(&d_data[gpu], N_PER_GPU * sizeof(float));
        cudaMalloc(&d_partial[gpu], NUM_BLOCKS * sizeof(float));
        cudaStreamCreate(&streams[gpu]);
        
        // Init with 1s
        float* h_temp = new float[N_PER_GPU];
        for (int i = 0; i < N_PER_GPU; i++) h_temp[i] = 1.0f;
        cudaMemcpy(d_data[gpu], h_temp, N_PER_GPU * sizeof(float),
                   cudaMemcpyHostToDevice);
        delete[] h_temp;
    }
    
    // ============================================
    // Phase 1: Reduce Within Each GPU
    // ============================================
    for (int gpu = 0; gpu < NUM_GPUS; gpu++) {
        cudaSetDevice(gpu);
        reduce<<<NUM_BLOCKS, BLOCK_SIZE, BLOCK_SIZE * sizeof(float), 
                 streams[gpu]>>>(d_partial[gpu], d_data[gpu], N_PER_GPU);
        
        // Second reduction pass
        int remaining = NUM_BLOCKS;
        while (remaining > 1) {
            int newBlocks = (remaining + BLOCK_SIZE - 1) / BLOCK_SIZE;
            reduce<<<newBlocks, BLOCK_SIZE, BLOCK_SIZE * sizeof(float),
                     streams[gpu]>>>(d_partial[gpu], d_partial[gpu], remaining);
            remaining = newBlocks;
        }
    }
    
    // ============================================
    // Phase 2: Collect Partial Sums to Host
    // ============================================
    for (int gpu = 0; gpu < NUM_GPUS; gpu++) {
        cudaSetDevice(gpu);
        cudaMemcpyAsync(&h_partial[gpu], d_partial[gpu], sizeof(float),
                        cudaMemcpyDeviceToHost, streams[gpu]);
    }
    
    // Sync all
    for (int gpu = 0; gpu < NUM_GPUS; gpu++) {
        cudaSetDevice(gpu);
        cudaStreamSynchronize(streams[gpu]);
    }
    
    // ============================================
    // Phase 3: Final Sum on Host
    // ============================================
    float total = 0.0f;
    for (int gpu = 0; gpu < NUM_GPUS; gpu++) {
        printf("GPU %d partial sum: %.0f\n", gpu, h_partial[gpu]);
        total += h_partial[gpu];
    }
    
    printf("Total sum: %.0f (expected %d)\n", total, TOTAL_N);
    
    // Cleanup
    for (int gpu = 0; gpu < NUM_GPUS; gpu++) {
        cudaSetDevice(gpu);
        cudaFree(d_data[gpu]);
        cudaFree(d_partial[gpu]);
        cudaStreamDestroy(streams[gpu]);
    }
    
    return 0;
}
```

### 🔷 CUDA C++ Implementation (Primary)

In [ ]:
%%writefile multi_gpu_reduction.cu
// multi_gpu_reduction.cu - Reduce across multiple GPUs
#include <stdio.h>
#include <cuda_runtime.h>

// Block reduction kernel
__global__ void reduce(float* out, float* in, int n) {
    extern __shared__ float sdata[];
    
    int tid = threadIdx.x;
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    
    sdata[tid] = (idx < n) ? in[idx] : 0.0f;
    __syncthreads();
    
    for (int s = blockDim.x / 2; s > 0; s >>= 1) {
        if (tid < s) {
            sdata[tid] += sdata[tid + s];
        }
        __syncthreads();
    }
    
    if (tid == 0) {
        out[blockIdx.x] = sdata[0];
    }
}

int main() {
    int deviceCount;
    cudaGetDeviceCount(&deviceCount);
    const int NUM_GPUS = min(deviceCount, 4);
    
    const int TOTAL_N = 1 << 24;
    const int N_PER_GPU = TOTAL_N / NUM_GPUS;
    const int BLOCK_SIZE = 256;
    const int NUM_BLOCKS = (N_PER_GPU + BLOCK_SIZE - 1) / BLOCK_SIZE;
    
    // Per-GPU data
    float* d_data[NUM_GPUS];
    float* d_partial[NUM_GPUS];
    float h_partial[NUM_GPUS];  // Final partial sums
    cudaStream_t streams[NUM_GPUS];
    
    // Allocate and initialize
    for (int gpu = 0; gpu < NUM_GPUS; gpu++) {
        cudaSetDevice(gpu);
        cudaMalloc(&d_data[gpu], N_PER_GPU * sizeof(float));
        cudaMalloc(&d_partial[gpu], NUM_BLOCKS * sizeof(float));
        cudaStreamCreate(&streams[gpu]);
        
        // Init with 1s
        float* h_temp = new float[N_PER_GPU];
        for (int i = 0; i < N_PER_GPU; i++) h_temp[i] = 1.0f;
        cudaMemcpy(d_data[gpu], h_temp, N_PER_GPU * sizeof(float),
                   cudaMemcpyHostToDevice);
        delete[] h_temp;
    }
    
    // ============================================
    // Phase 1: Reduce Within Each GPU
    // ============================================
    for (int gpu = 0; gpu < NUM_GPUS; gpu++) {
        cudaSetDevice(gpu);
        reduce<<<NUM_BLOCKS, BLOCK_SIZE, BLOCK_SIZE * sizeof(float), 
                 streams[gpu]>>>(d_partial[gpu], d_data[gpu], N_PER_GPU);
        
        // Second reduction pass
        int remaining = NUM_BLOCKS;
        while (remaining > 1) {
            int newBlocks = (remaining + BLOCK_SIZE - 1) / BLOCK_SIZE;
            reduce<<<newBlocks, BLOCK_SIZE, BLOCK_SIZE * sizeof(float),
                     streams[gpu]>>>(d_partial[gpu], d_partial[gpu], remaining);
            remaining = newBlocks;
        }
    }
    
    // ============================================
    // Phase 2: Collect Partial Sums to Host
    // ============================================
    for (int gpu = 0; gpu < NUM_GPUS; gpu++) {
        cudaSetDevice(gpu);
        cudaMemcpyAsync(&h_partial[gpu], d_partial[gpu], sizeof(float),
                        cudaMemcpyDeviceToHost, streams[gpu]);
    }
    
    // Sync all
    for (int gpu = 0; gpu < NUM_GPUS; gpu++) {
        cudaSetDevice(gpu);
        cudaStreamSynchronize(streams[gpu]);
    }
    
    // ============================================
    // Phase 3: Final Sum on Host
    // ============================================
    float total = 0.0f;
    for (int gpu = 0; gpu < NUM_GPUS; gpu++) {
        printf("GPU %d partial sum: %.0f\n", gpu, h_partial[gpu]);
        total += h_partial[gpu];
    }
    
    printf("Total sum: %.0f (expected %d)\n", total, TOTAL_N);
    
    // Cleanup
    for (int gpu = 0; gpu < NUM_GPUS; gpu++) {
        cudaSetDevice(gpu);
        cudaFree(d_data[gpu]);
        cudaFree(d_partial[gpu]);
        cudaStreamDestroy(streams[gpu]);
    }
    
    return 0;
}

In [ ]:
!nvcc -arch=sm_75 -o multi_gpu_reduction multi_gpu_reduction.cu
!./multi_gpu_reduction

---

## Part 4: Load Balancing

### Handling Non-Uniform Work

```
Static Partitioning (Simple but can be unbalanced):
━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━
GPU 0: [0, N/4)   → May be easy work
GPU 1: [N/4, N/2) → May be hard work
GPU 2: [N/2, 3N/4) → Variable
GPU 3: [3N/4, N)  → Variable

Dynamic Load Balancing:
━━━━━━━━━━━━━━━━━━━━━━━━
• Work queue with atomic counter
• GPUs grab chunks as they finish
• Better for non-uniform work
```

```cpp
// Simple work-stealing pattern
__device__ int workCounter = 0;

__global__ void dynamicWork(float* data, int totalChunks, int chunkSize) {
    while (true) {
        // Atomically get next chunk
        int myChunk = atomicAdd(&workCounter, 1);
        
        if (myChunk >= totalChunks) break;
        
        // Process chunk
        int start = myChunk * chunkSize;
        for (int i = threadIdx.x; i < chunkSize; i += blockDim.x) {
            data[start + i] = processElement(data[start + i]);
        }
    }
}
```

---

## 🎯 Exercises

### 🔷 CUDA C++ Exercises (Primary)

In [ ]:
%%writefile multi_gpu_patterns_exercises.cu
// multi_gpu_patterns_exercises.cu - Multi-GPU Pattern Exercises
#include <stdio.h>
#include <stdlib.h>
#include <cuda_runtime.h>

#define CHECK_CUDA(call) { \
    cudaError_t err = call; \
    if (err != cudaSuccess) { \
        printf("CUDA error %s:%d: %s\n", __FILE__, __LINE__, \
               cudaGetErrorString(err)); \
        exit(1); \
    } \
}

// ============================================
// Exercise 1: 2D Domain Decomposition
// ============================================
__global__ void matrixScaleKernel(float* matrix, int width, int height, float scale) {
    int x = blockIdx.x * blockDim.x + threadIdx.x;
    int y = blockIdx.y * blockDim.y + threadIdx.y;
    
    if (x < width && y < height) {
        int idx = y * width + x;
        matrix[idx] *= scale;
    }
}

void test2DDecomposition() {
    printf("\n=== Exercise 1: 2D Domain Decomposition ===\n");
    
    int deviceCount;
    CHECK_CUDA(cudaGetDeviceCount(&deviceCount));
    int numGPUs = (deviceCount >= 2) ? 2 : 1;
    
    const int WIDTH = 4096;
    const int HEIGHT = 4096;
    const size_t totalSize = WIDTH * HEIGHT * sizeof(float);
    
    printf("Matrix: %d x %d (%.0f MB)\n", WIDTH, HEIGHT, totalSize / (1024.0 * 1024.0));
    printf("Using %d GPU(s) with row decomposition\n\n", numGPUs);
    
    // Host matrix
    float* h_matrix;
    CHECK_CUDA(cudaMallocHost(&h_matrix, totalSize));
    
    // Initialize
    for (int i = 0; i < WIDTH * HEIGHT; i++) {
        h_matrix[i] = (float)(i % 100) / 100.0f;
    }
    
    // Calculate row chunks per GPU
    int rowsPerGPU = HEIGHT / numGPUs;
    int remainderRows = HEIGHT % numGPUs;
    
    float* d_chunks[2];
    cudaStream_t streams[2];
    int rowOffsets[2], rowCounts[2];
    
    cudaEvent_t start, stop;
    CHECK_CUDA(cudaSetDevice(0));
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    
    // Setup
    for (int i = 0; i < numGPUs; i++) {
        rowOffsets[i] = i * rowsPerGPU;
        rowCounts[i] = rowsPerGPU + (i == numGPUs - 1 ? remainderRows : 0);
        size_t chunkSize = rowCounts[i] * WIDTH * sizeof(float);
        
        CHECK_CUDA(cudaSetDevice(i));
        CHECK_CUDA(cudaStreamCreate(&streams[i]));
        CHECK_CUDA(cudaMalloc(&d_chunks[i], chunkSize));
        
        printf("GPU %d: rows %d-%d (%d rows, %.1f MB)\n",
               i, rowOffsets[i], rowOffsets[i] + rowCounts[i] - 1,
               rowCounts[i], chunkSize / (1024.0 * 1024.0));
    }
    
    cudaEventRecord(start);
    
    // Copy chunks to GPUs
    for (int i = 0; i < numGPUs; i++) {
        CHECK_CUDA(cudaSetDevice(i));
        size_t offset = rowOffsets[i] * WIDTH;
        size_t chunkSize = rowCounts[i] * WIDTH * sizeof(float);
        CHECK_CUDA(cudaMemcpyAsync(d_chunks[i], h_matrix + offset,
                                   chunkSize, cudaMemcpyHostToDevice, streams[i]));
    }
    
    // Process each chunk
    dim3 blockDim(16, 16);
    for (int i = 0; i < numGPUs; i++) {
        CHECK_CUDA(cudaSetDevice(i));
        dim3 gridDim((WIDTH + blockDim.x - 1) / blockDim.x,
                     (rowCounts[i] + blockDim.y - 1) / blockDim.y);
        matrixScaleKernel<<<gridDim, blockDim, 0, streams[i]>>>(
            d_chunks[i], WIDTH, rowCounts[i], 2.0f);
    }
    
    // Copy back
    for (int i = 0; i < numGPUs; i++) {
        CHECK_CUDA(cudaSetDevice(i));
        size_t offset = rowOffsets[i] * WIDTH;
        size_t chunkSize = rowCounts[i] * WIDTH * sizeof(float);
        CHECK_CUDA(cudaMemcpyAsync(h_matrix + offset, d_chunks[i],
                                   chunkSize, cudaMemcpyDeviceToHost, streams[i]));
    }
    
    // Sync
    for (int i = 0; i < numGPUs; i++) {
        CHECK_CUDA(cudaSetDevice(i));
        CHECK_CUDA(cudaStreamSynchronize(streams[i]));
    }
    
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    
    float ms;
    cudaEventElapsedTime(&ms, start, stop);
    
    // Verify (check a few samples)
    bool correct = true;
    for (int i = 0; i < 10; i++) {
        int idx = (i * WIDTH * HEIGHT / 10) % (WIDTH * HEIGHT);
        float expected = ((idx % 100) / 100.0f) * 2.0f;
        if (fabs(h_matrix[idx] - expected) > 1e-5) {
            printf("Mismatch at %d: %.4f vs %.4f\n", idx, h_matrix[idx], expected);
            correct = false;
        }
    }
    
    printf("\nTime: %.3f ms\n", ms);
    printf("Throughput: %.2f GB/s\n", 2.0 * totalSize / (ms / 1000.0) / (1024*1024*1024));
    printf("Result: %s\n", correct ? "CORRECT ✓" : "INCORRECT ✗");
    
    // Cleanup
    for (int i = 0; i < numGPUs; i++) {
        cudaSetDevice(i);
        cudaFree(d_chunks[i]);
        cudaStreamDestroy(streams[i]);
    }
    cudaFreeHost(h_matrix);
}

// ============================================
// Exercise 2: Ring AllReduce Sum
// ============================================
__global__ void partialSumKernel(float* data, float* result, int n) {
    extern __shared__ float sdata[];
    
    int tid = threadIdx.x;
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    
    sdata[tid] = (idx < n) ? data[idx] : 0.0f;
    __syncthreads();
    
    // Reduction in shared memory
    for (int s = blockDim.x / 2; s > 0; s >>= 1) {
        if (tid < s) {
            sdata[tid] += sdata[tid + s];
        }
        __syncthreads();
    }
    
    if (tid == 0) {
        atomicAdd(result, sdata[0]);
    }
}

void testRingAllReduce() {
    printf("\n=== Exercise 2: Ring AllReduce Sum ===\n");
    
    int deviceCount;
    CHECK_CUDA(cudaGetDeviceCount(&deviceCount));
    int numGPUs = (deviceCount >= 2) ? 2 : 1;
    
    const int N = 1024 * 1024;  // 1M elements per GPU
    printf("Each GPU has %d elements\n", N);
    printf("Using %d GPU(s)\n\n", numGPUs);
    
    // Allocate and initialize on each GPU
    float* d_data[2];
    float* d_partialSum[2];
    float h_partialSums[2];
    
    for (int i = 0; i < numGPUs; i++) {
        CHECK_CUDA(cudaSetDevice(i));
        CHECK_CUDA(cudaMalloc(&d_data[i], N * sizeof(float)));
        CHECK_CUDA(cudaMalloc(&d_partialSum[i], sizeof(float)));
        CHECK_CUDA(cudaMemset(d_partialSum[i], 0, sizeof(float)));
        
        // Initialize with GPU-specific pattern: GPU i has values i+1
        float* h_temp = (float*)malloc(N * sizeof(float));
        for (int j = 0; j < N; j++) {
            h_temp[j] = (float)(i + 1);
        }
        CHECK_CUDA(cudaMemcpy(d_data[i], h_temp, N * sizeof(float), cudaMemcpyHostToDevice));
        free(h_temp);
    }
    
    cudaEvent_t start, stop;
    CHECK_CUDA(cudaSetDevice(0));
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    
    cudaEventRecord(start);
    
    // Step 1: Local reduction on each GPU
    int blockSize = 256;
    int gridSize = (N + blockSize - 1) / blockSize;
    
    for (int i = 0; i < numGPUs; i++) {
        CHECK_CUDA(cudaSetDevice(i));
        partialSumKernel<<<gridSize, blockSize, blockSize * sizeof(float)>>>(
            d_data[i], d_partialSum[i], N);
    }
    
    // Sync all GPUs
    for (int i = 0; i < numGPUs; i++) {
        CHECK_CUDA(cudaSetDevice(i));
        CHECK_CUDA(cudaDeviceSynchronize());
    }
    
    // Step 2: Gather partial sums to host
    for (int i = 0; i < numGPUs; i++) {
        CHECK_CUDA(cudaSetDevice(i));
        CHECK_CUDA(cudaMemcpy(&h_partialSums[i], d_partialSum[i], 
                              sizeof(float), cudaMemcpyDeviceToHost));
    }
    
    // Step 3: Final reduction on host (or could be on GPU 0)
    float totalSum = 0;
    for (int i = 0; i < numGPUs; i++) {
        printf("GPU %d partial sum: %.0f\n", i, h_partialSums[i]);
        totalSum += h_partialSums[i];
    }
    
    // Step 4: Broadcast back (in ring pattern for multi-GPU)
    // For simplicity, we just compute expected
    float expectedSum = 0;
    for (int i = 0; i < numGPUs; i++) {
        expectedSum += (float)(i + 1) * N;  // Each GPU has N copies of (i+1)
    }
    
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    
    float ms;
    cudaEventElapsedTime(&ms, start, stop);
    
    printf("\nTotal sum: %.0f\n", totalSum);
    printf("Expected:  %.0f\n", expectedSum);
    printf("Time: %.3f ms\n", ms);
    printf("Result: %s\n", (totalSum == expectedSum) ? "CORRECT ✓" : "INCORRECT ✗");
    
    // Cleanup
    for (int i = 0; i < numGPUs; i++) {
        cudaSetDevice(i);
        cudaFree(d_data[i]);
        cudaFree(d_partialSum[i]);
    }
}

// ============================================
// Exercise 3: Async Halo Exchange + Compute
// ============================================
__global__ void stencilInteriorKernel(float* data, float* output, int width, int height) {
    // Process interior points (not touching boundaries)
    int x = blockIdx.x * blockDim.x + threadIdx.x + 1;  // Skip first column
    int y = blockIdx.y * blockDim.y + threadIdx.y + 1;  // Skip first row
    
    if (x < width - 1 && y < height - 1) {
        int idx = y * width + x;
        // 5-point stencil
        output[idx] = 0.2f * (data[idx] + 
                              data[idx - 1] + data[idx + 1] +
                              data[idx - width] + data[idx + width]);
    }
}

__global__ void stencilBoundaryKernel(float* data, float* output, int width, int height,
                                       int boundaryType) {
    // boundaryType: 0=top, 1=bottom, 2=left, 3=right
    int tid = blockIdx.x * blockDim.x + threadIdx.x;
    
    if (boundaryType == 0 && tid > 0 && tid < width - 1) {
        // Top row (y=1, after halo)
        int idx = width + tid;  // Row 1
        output[idx] = 0.2f * (data[idx] + data[idx-1] + data[idx+1] +
                              data[idx-width] + data[idx+width]);
    }
    else if (boundaryType == 1 && tid > 0 && tid < width - 1) {
        // Bottom row
        int y = height - 2;
        int idx = y * width + tid;
        output[idx] = 0.2f * (data[idx] + data[idx-1] + data[idx+1] +
                              data[idx-width] + data[idx+width]);
    }
}

void testAsyncHaloExchange() {
    printf("\n=== Exercise 3: Async Halo Exchange + Compute ===\n");
    
    int deviceCount;
    CHECK_CUDA(cudaGetDeviceCount(&deviceCount));
    
    // This pattern is best demonstrated on a single GPU with streams
    printf("Demonstrating overlapped halo exchange pattern on GPU 0\n\n");
    
    const int WIDTH = 1024;
    const int HEIGHT = 1024;
    const int HALO = 1;  // 1-cell halo for 5-point stencil
    
    // Simulate 2 domains (top and bottom halves)
    int halfHeight = HEIGHT / 2 + HALO;  // Include halo row
    size_t domainSize = WIDTH * halfHeight * sizeof(float);
    
    printf("Domain: %d x %d, split into 2 halves\n", WIDTH, HEIGHT);
    printf("Each half: %d x %d (including halo)\n\n", WIDTH, halfHeight);
    
    CHECK_CUDA(cudaSetDevice(0));
    
    float *d_domain0, *d_domain1;
    float *d_output0, *d_output1;
    CHECK_CUDA(cudaMalloc(&d_domain0, domainSize));
    CHECK_CUDA(cudaMalloc(&d_domain1, domainSize));
    CHECK_CUDA(cudaMalloc(&d_output0, domainSize));
    CHECK_CUDA(cudaMalloc(&d_output1, domainSize));
    
    // Initialize with pattern
    float* h_init = (float*)malloc(domainSize);
    for (int i = 0; i < WIDTH * halfHeight; i++) {
        h_init[i] = 1.0f;
    }
    CHECK_CUDA(cudaMemcpy(d_domain0, h_init, domainSize, cudaMemcpyHostToDevice));
    CHECK_CUDA(cudaMemcpy(d_domain1, h_init, domainSize, cudaMemcpyHostToDevice));
    
    cudaStream_t computeStream, haloStream;
    cudaStreamCreate(&computeStream);
    cudaStreamCreate(&haloStream);
    
    cudaEvent_t start, stop, haloComplete;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    cudaEventCreate(&haloComplete);
    
    cudaEventRecord(start);
    
    // === Overlapped execution pattern ===
    
    // 1. Start halo exchange (async copy of boundary rows)
    float* haloBuffer;
    CHECK_CUDA(cudaMallocHost(&haloBuffer, WIDTH * sizeof(float)));
    
    // Copy bottom row of domain0 to halo of domain1 (via host for demo)
    int srcRow = halfHeight - HALO - 1;  // Last non-halo row of domain0
    CHECK_CUDA(cudaMemcpyAsync(haloBuffer, d_domain0 + srcRow * WIDTH,
                                WIDTH * sizeof(float), cudaMemcpyDeviceToHost, haloStream));
    CHECK_CUDA(cudaMemcpyAsync(d_domain1, haloBuffer,  // First row (halo) of domain1
                                WIDTH * sizeof(float), cudaMemcpyHostToDevice, haloStream));
    cudaEventRecord(haloComplete, haloStream);
    
    // 2. Compute interior (can run in parallel with halo exchange)
    dim3 blockDim(16, 16);
    dim3 gridDim((WIDTH - 2 + blockDim.x - 1) / blockDim.x,
                 (halfHeight - 2 + blockDim.y - 1) / blockDim.y);
    
    stencilInteriorKernel<<<gridDim, blockDim, 0, computeStream>>>(
        d_domain0, d_output0, WIDTH, halfHeight);
    stencilInteriorKernel<<<gridDim, blockDim, 0, computeStream>>>(
        d_domain1, d_output1, WIDTH, halfHeight);
    
    // 3. Wait for halo exchange to complete
    cudaStreamWaitEvent(computeStream, haloComplete, 0);
    
    // 4. Process boundary rows (depend on halo data)
    int boundaryBlocks = (WIDTH + 255) / 256;
    stencilBoundaryKernel<<<boundaryBlocks, 256, 0, computeStream>>>(
        d_domain1, d_output1, WIDTH, halfHeight, 0);  // Top boundary of domain1
    
    cudaEventRecord(stop, computeStream);
    CHECK_CUDA(cudaStreamSynchronize(computeStream));
    
    float ms;
    cudaEventElapsedTime(&ms, start, stop);
    
    printf("Stencil computation with overlapped halo exchange:\n");
    printf("  Total time: %.3f ms\n", ms);
    printf("  Pattern: Halo copy || Interior compute -> Boundary compute\n");
    printf("\n✓ Demonstrated async halo exchange pattern\n");
    
    // Cleanup
    cudaStreamDestroy(computeStream);
    cudaStreamDestroy(haloStream);
    cudaFree(d_domain0);
    cudaFree(d_domain1);
    cudaFree(d_output0);
    cudaFree(d_output1);
    cudaFreeHost(haloBuffer);
    free(h_init);
}

// ============================================
// Main
// ============================================
int main() {
    printf("╔══════════════════════════════════════════════════════════════╗\n");
    printf("║           MULTI-GPU PATTERNS - EXERCISES                      ║\n");
    printf("╚══════════════════════════════════════════════════════════════╝\n");
    
    int deviceCount;
    CHECK_CUDA(cudaGetDeviceCount(&deviceCount));
    printf("\nFound %d GPU(s)\n", deviceCount);
    
    test2DDecomposition();
    testRingAllReduce();
    testAsyncHaloExchange();
    
    printf("\n✓ All exercises completed!\n");
    return 0;
}

In [ ]:
!nvcc -arch=sm_75 -o multi_gpu_patterns_exercises multi_gpu_patterns_exercises.cu && ./multi_gpu_patterns_exercises

### 🔶 Python/Numba Exercises (Optional)

### Exercise 1: 2D Decomposition
Implement 2D domain decomposition for a matrix operation.

### Exercise 2: Ring AllReduce
Implement ring allreduce pattern for multi-GPU sum.

### Exercise 3: Async Halo Exchange
Overlap halo exchange with interior computation.

---

## Key Takeaways

```
┌─────────────────────────────────────────────────────────────────┐
│                  MULTI-GPU PATTERNS                             │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│  🏭 THE FACTORY MANAGER MENTAL MODEL:                           │
│  • You're the manager coordinating multiple factories (GPUs)    │
│  • Each factory works independently on its chunk                │
│  • Boundaries require communication (halo exchange)             │
│  • Results need combination (reduction)                         │
│                                                                 │
├─────────────────────────────────────────────────────────────────┤
│  📊 DOMAIN DECOMPOSITION:                                       │
│                                                                 │
│  int chunkSize = N / numGPUs;                                   │
│  int remainder = N % numGPUs;                                   │
│  // GPU i gets: chunkSize + (i < remainder ? 1 : 0)             │
│                                                                 │
│  1D: Simple array split                                         │
│  2D: Row-wise, column-wise, or tile-based                       │
│  3D: Slab or cube decomposition                                 │
│                                                                 │
├─────────────────────────────────────────────────────────────────┤
│  🔄 HALO EXCHANGE (Stencil Operations):                         │
│                                                                 │
│  Pattern:                                                       │
│  1. Allocate: realData + 2*haloSize                             │
│  2. Copy: exchange boundaries with neighbors                    │
│  3. Compute: include halo in stencil                            │
│  4. Repeat: if iterative algorithm                              │
│                                                                 │
│  ⚡ Optimization: Overlap halo copy with interior compute!      │
│                                                                 │
├─────────────────────────────────────────────────────────────────┤
│  ➕ MULTI-GPU REDUCTION:                                        │
│                                                                 │
│  Strategy 1: Collect to Host                                    │
│    Each GPU → partial sum → host combines                       │
│                                                                 │
│  Strategy 2: Collect to GPU 0                                   │
│    Each GPU → partial sum → P2P to GPU 0 → GPU 0 combines       │
│                                                                 │
│  Strategy 3: Ring AllReduce (advanced)                          │
│    Circular passing + accumulation - all GPUs get result        │
│                                                                 │
├─────────────────────────────────────────────────────────────────┤
│  ⚖️ LOAD BALANCING:                                             │
│                                                                 │
│  Static: Simple, works for uniform work                         │
│  Dynamic: Work queues for variable workloads (GPU grabs tasks)  │
│                                                                 │
│  💡 Callback to Week 7: Remember occupancy? Now think about     │
│  "multi-GPU occupancy" - keeping ALL GPUs busy!                 │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

---

## What's Next?

You've learned to coordinate multiple factories. **Day 3** is your **comprehensive optimization review** - a full performance tuning checklist covering everything from Week 1 to now!

Think of it as your quality control manual before the final capstone project. 📋✓